In [1]:
import numpy as np
from time import time
from torch.utils.data import DataLoader 

from data_utils import prepare_formated_data
from vectorrepr.datasets.timeseries import TimeSeriesDataset


2025-05-10 22:15:31.970 | INFO     | vectorrepr.config:<module>:11 - PROJ_ROOT path is: /Users/ytqiang/workspace/vectorrepr


##### 准备数据文件

In [2]:
df = prepare_formated_data(start_date="2023-01-01", end_date="2024-12-31", stock_datapath="../data/external/stock.parquet.gz")
df.to_parquet("../data/processed/stock500_2023_2024_ts.parquet")
df = prepare_formated_data(start_date="2025-01-01", end_date="2025-04-30", stock_datapath="../data/external/stock.parquet.gz")
df.to_parquet("../data/processed/stock500_20250101_20250430_ts.parquet")

2025-05-10 22:16:48.530 | INFO     | data_utils:prepare_formated_data:24 - Data prepared: Data shape: (278662, 9)
2025-05-10 22:18:04.278 | INFO     | data_utils:prepare_formated_data:24 - Data prepared: Data shape: (40743, 9)


##### 持久化中间数据，方便后面使用

In [3]:
def prepare_interim_data(dataset: TimeSeriesDataset):

    previous_increase_ratio_list = []
    post_increase_ratio_list = []
    samples = []
    labels = []

    dataloader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=4)
    for i, (sample, label) in enumerate(dataloader):
        previous_increase_ratio_list.append(sample[:, :, 3]/sample[:, 0, 3:4])
        post_increase_ratio_list.append(label[:, :, 1] / sample[:, -1, 3:4])
        samples.append(sample)
        labels.append(label)
        if i % 100 == 0:
            print(i)
    samples = np.concatenate(samples, axis=0)
    labels = np.concatenate(labels, axis=0)
    previous_increase_ratio = np.concatenate(previous_increase_ratio_list, axis=0)
    post_increase_ratio = np.concatenate(post_increase_ratio_list, axis=0)

    return previous_increase_ratio, post_increase_ratio, samples, labels

In [5]:
# train_dataset = TimeSeriesDataset(
#     "../data/processed/stock500_2023_2024_ts.parquet",
#     time_idx="DateIdx",
#     group_ids="Ticker",
#     feature_columns=["Open", "High", "Low", "Close", "Volume", "Adj Close"],
#     na_handling=-1,
#     input_steps=30,
#     predict_steps=5,
# )
# previous_increase_ratio, post_increase_ratio, samples, labels = prepare_interim_data(train_dataset)
# print("train data prepared")
# np.savez_compressed("../data/interim/stock500_2023_2024_samples.npz", samples=samples)
# np.savez_compressed("../data/interim/stock500_2023_2024_labels.npz", labels=labels)
# np.savez_compressed("../data/interim/stock500_2023_2024_previous_increase_ratio.npz", previous_increase_ratio=previous_increase_ratio)
# np.savez_compressed("../data/interim/stock500_2023_2024_post_increase_ratio.npz", post_increase_ratio=post_increase_ratio)

test_dataset = TimeSeriesDataset(
    "../data/processed/stock500_20250101_20250430_ts.parquet",
    time_idx="DateIdx",
    group_ids="Ticker",
    feature_columns=["Open", "High", "Low", "Close", "Volume", "Adj Close"],
    na_handling=-1,
    input_steps=30,
    predict_steps=5,
)
previous_increase_ratio, post_increase_ratio, samples, labels = prepare_interim_data(test_dataset)
print("test data prepared")
np.savez_compressed("../data/interim/stock500_20250101_20250430_samples.npz", data=samples)
np.savez_compressed("../data/interim/stock500_20250101_20250430_labels.npz", data=labels)
np.savez_compressed("../data/interim/stock500_20250101_20250430_previous_increase_ratio.npz", previous_increase_ratio=previous_increase_ratio)
np.savez_compressed("../data/interim/stock500_20250101_20250430_post_increase_ratio.npz", post_increase_ratio=post_increase_ratio)


2025-05-10 23:06:17.924 | INFO     | vectorrepr.config:<module>:11 - PROJ_ROOT path is: /Users/ytqiang/workspace/vectorrepr
2025-05-10 23:06:20.670 | INFO     | vectorrepr.config:<module>:11 - PROJ_ROOT path is: /Users/ytqiang/workspace/vectorrepr
2025-05-10 23:06:23.258 | INFO     | vectorrepr.config:<module>:11 - PROJ_ROOT path is: /Users/ytqiang/workspace/vectorrepr
2025-05-10 23:06:26.061 | INFO     | vectorrepr.config:<module>:11 - PROJ_ROOT path is: /Users/ytqiang/workspace/vectorrepr


0
100
200
test data prepared
